In [ ]:
!pip install pdfplumber pillow opencv-python pytesseract pandas

In [20]:
import pdfplumber
from PIL import Image

# Open the PDF file
pdf_path = '/mnt/data/Original Files.pdf'
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        # Convert PDF page to image
        page_image = page.to_image(resolution=300)
        # Save the image
        page_image.save(f'page_{i}.png')

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Original Files.pdf'

In [21]:
import cv2

def preprocess_image(image_path):
    # Load an image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Apply edge detection
    edges = cv2.Canny(thresh, 50, 150)

    return edges

# Preprocess the first image for demonstration
preprocessed_image = preprocess_image('page_0.png')
cv2.imwrite('preprocessed_page_0.png', preprocessed_image)

True

In [22]:
import pytesseract

def extract_text(image_path):
    # Load and preprocess the image
    image = preprocess_image(image_path)

    # Extract text using PyTesseract
    text = pytesseract.image_to_string(image, config="--psm 6")
    return text

# Extract text from the first image for demonstration
text = extract_text('page_0.png')
print(text)

N@MESs
1. ALL STEEL SHALL BE GALVANIZED.
fo vo KO, L
/ 4 ‘\
< \A ONG? a
,] <—
va aw wa A
- 2, DET
f | 40
| /
l ( S-5201
| | IN i
(
S —|. — =
( N
( Nw
: | Dy
= = '%
fe) s<| | w
() NN iA
=| | \
. | N
ns | N a
2 (4
| NN ‘
A
DET ~ L.
\ a A = “a ‘\
! / / q (B)
/ $5201 o NC
[ W16x40 { ~ ” =
po Be a fo oe oe F555 555555 55 555 P55 5555 55 S559 55 5555554] 2
—— cb
= BE ww ! Z Z
V7 4 — | (P) — [ — | | ZO
_ [ Ry e « ZB — LE 2) |
< — | Oise | Sy | Ze Ne
8 re Fe = ad GE &
@ Sa, s<| | ale = =| | a <| | Zee ©
= | — io) OnA — & BZ S| |
— | WA = | | i ( L
| SS = WA = Say = BZ |
i Say | A | ey l Ae [
= <S —, I] a
W16x40
(x \ S-5201
| |
| |
8'-o” 8'-9Q” [ 8'-o” 8'-9Q”
<———<<<——- —— ——————q— 7
THESE PLANS ARE RELEASED FOR THE PURPOSE
q 00
SSE OF PROVIDING INTERIM REVIEW AND PRE-
CONSTRUCTION INFORMATION ONLY, UNDER THE
AUTHORITY OF FRED. E. FLESCH
PE# 54017 ON 04/25/24 THESE PLANS ARE
[2A /a\ yu @ TOS ps/L “| Devo Need One of These (1) NOT TO BE USED FOR CONSTRUCTION PURPOSES.
D fwd oe it ag
SwAles 12." 

In [30]:
def detect_shapes(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    preprocessed_image = preprocess_image(image_path)

    # Find contours
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours and classify shapes
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) == 4:
            # Rectangle (could be a wall, door, or window)
            x, y, w, h = cv2.boundingRect(approx)
            cv2.drawContours(image, [approx], 0, (0, 255, 0), 2)
            cv2.putText(image, "Rectangle", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the image with detected shapes
    cv2.imwrite('detected_shapes_page_0.png', image)

# Detect shapes in the first image for demonstration
detect_shapes('preprocessed_page_0.png')

In [26]:
import re

def identify_components(text, contours):
    components = []
    
    # Example pattern matching for common components (e.g., W sections, L sections)
    patterns = {
        'W_section': r'W\d+x\d+',
        'L_section': r'L\d+x\d+x\d+'
    }

    for line in text.split('\n'):
        for component_type, pattern in patterns.items():
            matches = re.findall(pattern, line)
            for match in matches:
                components.append({'type': component_type, 'label': match})

    # Further refine by checking contours (to be detailed based on specific needs)
    # For demonstration, we just list the detected components

    return components

# Identify components from the text of the first image
components = identify_components(text, contours)
print(components)

[{'type': 'W_section', 'label': 'W16x40'}, {'type': 'W_section', 'label': 'W16x40'}]


In [28]:
import pandas as pd

def generate_takeoff_list(components):
    takeoff_data = {
        'Component': [],
        'Type': [],
        'Quantity': [],
        'Unit': []
    }

    for component in components:
        takeoff_data['Component'].append(component['label'])
        takeoff_data['Type'].append(component['type'])
        takeoff_data['Quantity'].append(1)  # Placeholder, quantity logic to be implemented
        takeoff_data['Unit'].append('unit')  # Placeholder, unit logic to be implemented

    df = pd.DataFrame(takeoff_data)
    return df

# Generate the takeoff list DataFrame
df = generate_takeoff_list(components)
print(df)

  Component       Type  Quantity  Unit
0    W16x40  W_section         1  unit
1    W16x40  W_section         1  unit
